In [ ]:
import pandas as pd
import os
import time
import keras_ocr
from matplotlib import pyplot as plt
from matplotlib import image as mpimg

In [ ]:
pipeline = keras_ocr.pipeline.Pipeline()

In [ ]:
subdir_path = 'L:/DATA/Alouette_I/BATCH_II_Run1/04_processed/R014207817/4163-03A/'

In [ ]:
img_fns = []
for file in os.listdir(subdir_path):
    img_fns.append(subdir_path + file)

In [ ]:
batch_i = 4
batch_f = 6

In [ ]:
start = time.time()

imgs_ocr = [
    keras_ocr.tools.read(img) for img in img_fns[batch_i:batch_f]
]

end = time.time()
t = end - start
print('Time to read all images in subdirectory: ' + str(round(t/60, 1)) + ' min')

In [ ]:
start = time.time()
prediction_groups = pipeline.recognize(img_fns[batch_i:batch_f])
end = time.time()
t = end - start
print('Time to ocr process all images in subdirectory: ' + str(round(t/60, 1)) + ' min')

In [ ]:
len(prediction_groups)

In [ ]:
df_results = pd.DataFrame()
for i in range(0, len(prediction_groups)):
    df_ocr = pd.DataFrame()
    predicted_image = prediction_groups[i]
    if len(predicted_image) > 0:
        for text, box in predicted_image:
            row = pd.DataFrame({
                'number': text,
                'x': box[1][0],
                'y': box[1][1]
            }, index=[0])
            df_ocr = pd.concat([df_ocr, row])
        df_ocr = df_ocr.sort_values('x').reset_index(drop=True)
    if len(df_ocr) == 6:
        if df_ocr['number'].iloc[0] == '10':
            row2 = pd.DataFrame({
                'station_number': df_ocr['number'].iloc[1],
                'year': df_ocr['number'].iloc[2],
                'day_of_year': df_ocr['number'].iloc[3],
                'hour': df_ocr['number'].iloc[4][0:2],
                'minute': df_ocr['number'].iloc[4][2:],
                'second': df_ocr['number'].iloc[5],
                'filename': img_fns[batch_i + i].replace(subdir_path, '')
            }, index=[0])
            df_results = pd.concat([df_results, row2])

In [ ]:
df_results

In [ ]:
img_fn = subdir_path + '102.png'
img = mpimg.imread(img_fn)
plt.imshow(img)

#  

Clearly the default recognizer is not adequate, as it can sometimes classify fuzzy numbers as letters.

We will need to 'fine-tune' the recognizer to only recognize numbers:

Ref: https://keras-ocr.readthedocs.io/en/latest/examples/fine_tuning_recognizer.html <br>
Ref (better): https://keras-ocr.readthedocs.io/en/stable/examples/end_to_end_training.html <br>
Ref: https://github.com/faustomorales/keras-ocr/issues/54

In [41]:
import zipfile
import datetime
import string
import math
import os
import numpy as np

import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn.model_selection

import keras_ocr

#assert tf.test.is_gpu_available(), 'No GPU is available.'

In [42]:
data_dir = 'C:/users/rnaidoo/Documents/Projects_data/Alouette_I/keras_ocr/'
alphabet = string.digits + ''  #+ string.ascii_letters + '!?. '
recognizer_alphabet = ''.join(sorted(set(alphabet.lower())))
fonts = keras_ocr.data_generation.get_fonts(
    alphabet=alphabet,
    cache_dir=data_dir
)

Looking for C:/users/rnaidoo/Documents/Projects_data/Alouette_I/keras_ocr/fonts.zip


Filtering fonts.: 100%|██████████| 2746/2746 [00:12<00:00, 212.95it/s]


In [53]:
'''backgrounds = []
for i in range(0, 100):
    backgrounds.append(data_dir+'backgrounds_Alouette_I/18.jpg')
    backgrounds.append(data_dir+'backgrounds_Alouette_I/47.jpg')
    backgrounds.append(data_dir+'backgrounds_Alouette_I/65.jpg')'''

#backgrounds = keras_ocr.data_generation.get_backgrounds(cache_dir=data_dir)

Looking for C:/users/rnaidoo/Documents/Projects_data/Alouette_I/keras_ocr/backgrounds.zip


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\users\\rnaidoo\\Documents\\Projects_data\\Alouette_I\\keras_ocr\\backgrounds\\1024px-%d0%9a%d1%80%d0%b8%d1%81%d1%82%d0%b0%d0%bb%d0%bb%d1%8b_%d0%b2_%d0%b2%d1%8b%d1%81%d0%be%d1%85%d1%88%d0%b5%d0%b9_%d0%ba%d0%b0%d0%bf%d0%bb%d0%b5_%d0%9a%d0%be%d0%ba%d0%b0_%d0%9a%d0%be%d0%bb%d1%8b.jpg'

In [54]:
backgrounds = os.listdir(data_dir + 'backgrounds_KerasOCR')

In [55]:
text_generator = keras_ocr.data_generation.get_text_generator(alphabet=alphabet)
print('The first generated text is:', next(text_generator))

The first generated text is: 


In [56]:
def get_train_val_test_split(arr):
    train, valtest = sklearn.model_selection.train_test_split(arr, train_size=0.8, random_state=42)
    val, test = sklearn.model_selection.train_test_split(valtest, train_size=0.5, random_state=42)
    return train, val, test

In [57]:
#backgrounds = np.argmax(backgrounds, axis=0)
background_splits = get_train_val_test_split(backgrounds)
font_splits = get_train_val_test_split(fonts)

image_generators = [
    keras_ocr.data_generation.get_image_generator(
        height=640,
        width=640,
        text_generator=text_generator,
        font_groups={
            alphabet: current_fonts
        },
        backgrounds=current_backgrounds,
        font_size=(60, 120),
        margin=50,
        rotationX=(-0.05, 0.05),
        rotationY=(-0.05, 0.05),
        rotationZ=(-15, 15)
    )  for current_fonts, current_backgrounds in zip(
        font_splits,
        background_splits
    )
]

In [24]:
background_splits

(['C:/users/rnaidoo/Documents/Projects_data/Alouette_I/keras_ocr/backgrounds_Alouette_I/65.jpg',
  'C:/users/rnaidoo/Documents/Projects_data/Alouette_I/keras_ocr/backgrounds_Alouette_I/18.jpg',
  'C:/users/rnaidoo/Documents/Projects_data/Alouette_I/keras_ocr/backgrounds_Alouette_I/65.jpg',
  'C:/users/rnaidoo/Documents/Projects_data/Alouette_I/keras_ocr/backgrounds_Alouette_I/65.jpg',
  'C:/users/rnaidoo/Documents/Projects_data/Alouette_I/keras_ocr/backgrounds_Alouette_I/47.jpg',
  'C:/users/rnaidoo/Documents/Projects_data/Alouette_I/keras_ocr/backgrounds_Alouette_I/18.jpg',
  'C:/users/rnaidoo/Documents/Projects_data/Alouette_I/keras_ocr/backgrounds_Alouette_I/18.jpg'],
 ['C:/users/rnaidoo/Documents/Projects_data/Alouette_I/keras_ocr/backgrounds_Alouette_I/47.jpg'],
 ['C:/users/rnaidoo/Documents/Projects_data/Alouette_I/keras_ocr/backgrounds_Alouette_I/47.jpg'])

In [25]:
image_generators

[<generator object get_image_generator at 0x0000021AFFBA7270>,
 <generator object get_image_generator at 0x0000021AFFBA7190>,
 <generator object get_image_generator at 0x0000021AFFBA70B0>]

In [26]:
image_generators[1]

<generator object get_image_generator at 0x0000021AFFBA7190>

In [28]:
next(image_generators[1], image_generators[1])

<generator object get_image_generator at 0x0000021AFFBA7190>

In [61]:
# See what the first validation image looks like.
image, lines = next(image_generators[1])
text = keras_ocr.data_generation.convert_lines_to_paragraph(lines)
print('The first generated validation image (below) contains:', text)
plt.imshow(image)

StopIteration: 

In [62]:
detector = keras_ocr.detection.Detector(weights='clovaai_general')
recognizer = keras_ocr.recognition.Recognizer(
    alphabet=recognizer_alphabet,
    weights='kurapan'
)
recognizer.compile()
for layer in recognizer.backbone.layers:
    layer.trainable = False

Looking for C:\Users\rnaidoo\.keras-ocr\craft_mlt_25k.h5
Provided alphabet does not match pretrained alphabet. Using backbone weights only.
Looking for C:\Users\rnaidoo\.keras-ocr\crnn_kurapan_notop.h5


In [65]:
detector_batch_size = 1
detector_basepath = os.path.join(data_dir, f'detector_{datetime.datetime.now().isoformat()}')
detection_train_generator, detection_val_generator, detection_test_generator = [
    detector.get_batch_generator(
        image_generator=image_generator,
        batch_size=detector_batch_size
    ) for image_generator in image_generators
]
detector.model.fit(
    detection_train_generator,
    steps_per_epoch=math.ceil(3/detector_batch_size),
    epochs=1000,
    workers=0,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(restore_best_weights=True, patience=5),
        tf.keras.callbacks.CSVLogger(f'{detector_basepath}.csv'),
        tf.keras.callbacks.ModelCheckpoint(filepath=f'{detector_basepath}.h5')
    ],
    validation_data=detection_val_generator,
    validation_steps=math.ceil(3/detector_batch_size),
    batch_size=detector_batch_size
)

RuntimeError: generator raised StopIteration

#  

In [ ]:
recognizer = keras_ocr.recognition.Recognizer(alphabet=alphabet, weights=None)#'kurapan')
recognizer.compile()

In [ ]:
#predicted = recognizer.recognize(subdir_path + '102.png')
#predicted

Use 'Born Digital' training set to re-train model with new alphabet:

In [ ]:
train_labels = keras_ocr.datasets.get_born_digital_recognizer_dataset(
    split='train',
    cache_dir='.'
)
test_labels = keras_ocr.datasets.get_born_digital_recognizer_dataset(
    split='test',
    cache_dir='.'
)
train_labels = [(filepath, box, word.lower()) for filepath, box, word in train_labels]
test_labels = [(filepath, box, word.lower()) for filepath, box, word in test_labels]

In [ ]:
batch_size = 8
augmenter = imgaug.augmenters.Sequential([
    imgaug.augmenters.GammaContrast(gamma=(0.25, 3.0)),
])

train_labels, validation_labels = sklearn.model_selection.train_test_split(train_labels, test_size=0.2, random_state=42)
(training_image_gen, training_steps), (validation_image_gen, validation_steps) = [
    (
        keras_ocr.datasets.get_recognizer_image_generator(
            labels=labels,
            height=recognizer.model.input_shape[1],
            width=recognizer.model.input_shape[2],
            alphabet=alphabet,
            augmenter=augmenter
        ),
        len(labels) // batch_size
    ) for labels, augmenter in [(train_labels, augmenter), (validation_labels, None)]
]
training_gen, validation_gen = [
    recognizer.get_batch_generator(
        image_generator=image_generator,
        batch_size=batch_size
    )
    for image_generator in [training_image_gen, validation_image_gen]
]

In [ ]:
image, text = next(training_image_gen)
print('text:', text)
plt.imshow(image)

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, restore_best_weights=False),
    tf.keras.callbacks.ModelCheckpoint('recognizer_borndigital.h5', monitor='val_loss', save_best_only=True),
    tf.keras.callbacks.CSVLogger('recognizer_borndigital.csv')
]
recognizer.model.fit(
    training_gen,
    steps_per_epoch=training_steps,
    validation_steps=validation_steps,
    validation_data=validation_gen,
    callbacks=callbacks,
    epochs=1000,
)